In [1]:
import os
import pandas as pd
import csv
import numpy as np
import natsort
import matplotlib.pyplot as plt
from scipy import interpolate
import math

In [2]:
def alignwind(wind_df):
    Ub = wind_df['u'].mean()
    Vb = wind_df['v'].mean()
    Wb = wind_df['w'].mean()
    Sb = math.sqrt((Ub**2)+(Vb**2))
    beta = math.atan2(Wb,Sb)
    alpha = math.atan2(Vb,Ub)
    x1 = wind_df.index
    x = np.array(x1)
    Ur = wind_df['u']*math.cos(alpha)*math.cos(beta)+wind_df['v']*math.sin(alpha)*math.cos(beta)+wind_df['w']*math.sin(beta)
    Ur_arr = np.array(Ur)
    Vr = wind_df['u']*(-1)*math.sin(alpha)+wind_df['v']*math.cos(alpha)
    Vr_arr = np.array(Vr)
    Wr = wind_df['u']*(-1)*math.cos(alpha)*math.sin(beta)+wind_df['v']*(-1)*math.sin(alpha)*math.sin(beta)+wind_df['w']*math.cos(beta)     
    Wr_arr = np.array(Wr)
    T_arr = np.array(wind_df['T'])
    u_arr = np.array(wind_df['u'])
    v_arr = np.array(wind_df['v'])
    w_arr = np.array(wind_df['w'])
    
    fu_p = interpolate.interp1d(x, Ur_arr,fill_value="extrapolate") #interpolate u_p
    fv_p = interpolate.interp1d(x, Vr_arr,fill_value="extrapolate") #interpolate v_p
    fw_p = interpolate.interp1d(x, Wr_arr,fill_value="extrapolate") #interpolate w_p
    ft = interpolate.interp1d(x, T_arr,fill_value="extrapolate") #interpolate T (aka T_p)
    fu = interpolate.interp1d(x, u_arr,fill_value="extrapolate") #interpolate u
    fv = interpolate.interp1d(x, v_arr,fill_value="extrapolate") #interpolate v
    fw = interpolate.interp1d(x, w_arr,fill_value="extrapolate") #interpolate w
    
    RMY_xnew = np.arange(0, 24000)
    RMY_ynew_up = fu_p(RMY_xnew)   # use interpolation function returned by `interp1d`
    RMY_ynew_vp = fv_p(RMY_xnew)
    RMY_ynew_wp = fw_p(RMY_xnew)
    RMY_ynew_t = ft(RMY_xnew)
    RMY_ynew_u = fu(RMY_xnew)
    RMY_ynew_v = fv(RMY_xnew)
    RMY_ynew_w = fw(RMY_xnew)
    
    S = Sb
    b = beta*180/math.pi
    a = alpha*180/math.pi
    df = pd.DataFrame({'base_index':RMY_xnew,'u':RMY_ynew_u, 'v':RMY_ynew_v, 'w':RMY_ynew_u,'u_p':RMY_ynew_up, 'v_p':RMY_ynew_vp, 'w_p':RMY_ynew_wp, 'T':RMY_ynew_t})
    return S,b,a,df

In [3]:
def makeDF(filepath):
    RMY_arr = [] 
    RMYa_arr = []
    RMYb_arr = []
    RMYSb_arr = []

    WM1_arr = [] 
    WM1a_arr = []
    WM1b_arr = []
    WM1Sb_arr = []

    WM2_arr = [] 
    WM2a_arr = []
    WM2b_arr = []
    WM2Sb_arr = []

    R3_arr = [] 
    R3a_arr = []
    R3b_arr = []
    R3Sb_arr = []

    for root, dirnames, filenames in os.walk(filepath):
        for filename in natsort.natsorted(filenames):
            file = os.path.join(root, filename)

            if file.endswith(".S07"):
                RMY_df = pd.read_fwf(file, index_col=None, header = None)
                RMY_df.columns =['u', 'v', 'w', 'T', 'err_code']
                S,b,a,df = alignwind(RMY_df)
                RMY0 = df.mean(axis=0)
                RMY = np.transpose(RMY0)
                RMY_arr.append(RMY)
                RMYa_arr.append(a)
                RMYb_arr.append(b)
                RMYSb_arr.append(S)

            elif file.endswith(".S08"):
                WM1_df=pd.read_csv(file, index_col=None, header=None)
                WM1_df.columns = ['node_ad', 'u', 'v', 'w', 'units',"T", 'err_code','chk_sum']
                S,b,a,df = alignwind(WM1_df)
                WM1_0 = df.mean(axis=0)
                WM1 = np.transpose(WM1_0)
                WM1_arr.append(WM1)
                WM1a_arr.append(a)
                WM1b_arr.append(b)
                WM1Sb_arr.append(S)

            elif file.endswith(".S09"):
# for WM2
#                 WM2_df=pd.read_csv(file, index_col=None, header=None)
#                 WM2_df.columns = ['node_ad', 'u', 'v', 'w', 'units',"T", 'err_code','chk_sum']
#                 S,b,a,df = alignwind(WM2_df)
#                 WM2_0 = df.mean(axis=0)
#                 WM2 = np.transpose(WM2_0)
#                 WM2_arr.append(WM2)
#                 WM2a_arr.append(a)
#                 WM2b_arr.append(b)
#                 WM2Sb_arr.append(S)
# for CSAT
                WM2_df=pd.read_csv(file, index_col=None, header=None)
                WM2_df.columns = ['u', 'v', 'w', "T", '?']
                S,b,a,df = alignwind(WM2_df)
                WM2_0 = df.mean(axis=0)
                WM2 = np.transpose(WM2_0)
                WM2_arr.append(WM2)
                WM2a_arr.append(a)
                WM2b_arr.append(b)
                WM2Sb_arr.append(S)

            elif file.endswith(".S10"):
                R3_df=pd.read_csv(file, index_col=None, header=None)
                R3_df.columns = ['col1','col2', 'u', 'v', 'w', 'T','chk_sum']
                S,b,a,df = alignwind(R3_df)
                R3_0 = df.mean(axis=0)
                R3 = np.transpose(R3_0)
                R3_arr.append(R3)
                R3a_arr.append(a)
                R3b_arr.append(b)
                R3Sb_arr.append(S)
            else:
                continue

    return R3_arr,RMY_arr,WM1_arr,WM2_arr,R3a_arr,RMYa_arr,WM1a_arr,WM2a_arr,R3b_arr,RMYb_arr,WM1b_arr,WM2b_arr,R3Sb_arr,RMYSb_arr,WM1Sb_arr,WM2Sb_arr

In [11]:
R3_arr,RMY_arr,WM1_arr,WM2_arr,R3a_arr,RMYa_arr,WM1a_arr,WM2a_arr,R3b_arr,RMYb_arr,WM1b_arr,WM2b_arr,R3Sb_arr,RMYSb_arr,WM1Sb_arr,WM2Sb_arr = makeDF(r"F:\benchTest_data2\analysis_copy")
print("done")

done


In [6]:
R3_final = pd.concat(R3_arr, axis=1, ignore_index=True)
R3_finT = R3_final.T
R3_finT.columns =['base_index','R3_u', 'R3_v', 'R3_w', 'R3_u_', 'R3_v_', 'R3_w_', 'R3_T_']

RMY_final = pd.concat(RMY_arr, axis=1, ignore_index=True)
RMY_finT = RMY_final.T
RMY_finT.columns =['base_index','RMY_u', 'RMY_v', 'RMY_w', 'RMY_u_', 'RMY_v_', 'RMY_w_', 'RMY_T_']

WM1_final = pd.concat(WM1_arr, axis=1, ignore_index=True)
WM1_finT = WM1_final.T
WM1_finT.columns =['base_index','WM1_u', 'WM1_v', 'WM1_w', 'WM1_u_', 'WM1_v_', 'WM1_w_', 'WM1_T_']

WM2_final = pd.concat(WM2_arr, axis=1,ignore_index=True)
WM2_finT = WM2_final.T
WM2_finT.columns =['base_index','WM2_u', 'WM2_v', 'WM2_w', 'WM2_u_', 'WM2_v_', 'WM2_w_', 'WM2_T_']
#WM2_finT.columns =['u_', 'v_', 'w_', 'T_']

In [7]:
R3a_arr1 = np.array(R3a_arr)
R3b_arr1 = np.array(R3b_arr)
R3Sb_arr1 = np.array(R3Sb_arr)

RMYa_arr1 = np.array(RMYa_arr)
RMYb_arr1 = np.array(RMYb_arr)
RMYSb_arr1 = np.array(RMYSb_arr)

WM1a_arr1 = np.array(WM1a_arr)
WM1b_arr1 = np.array(WM1b_arr)
WM1Sb_arr1 = np.array(WM1Sb_arr)

WM2a_arr1 = np.array(WM2a_arr)
WM2b_arr1 = np.array(WM2b_arr)
WM2Sb_arr1 = np.array(WM2Sb_arr)

In [8]:
#alter Alpha to be 0-360 degrees (intead of -180 to +180)
R3a_360 = []
for i in range(len(R3a_arr1)):
    if R3a_arr1[i]<0:
        a_360_i = 360 + R3a_arr1[i]
    else:
        a_360_i = R3a_arr1[i]
    R3a_360.append(a_360_i)
R3_a360 = np.array(R3a_360)

RMYa_360 = []
for i in range(len(RMYa_arr1)):
    if RMYa_arr1[i]<0:
        a_360_i = 360 + RMYa_arr1[i]
    else:
        a_360_i = RMYa_arr1[i]
    RMYa_360.append(a_360_i)
RMY_a360 = np.array(RMYa_360)

WM1a_360 = []
for i in range(len(WM1a_arr1)):
    if WM1a_arr1[i]<0:
        a_360_i = 360 + WM1a_arr1[i]
    else:
        a_360_i = WM1a_arr1[i]
    WM1a_360.append(a_360_i)
WM1_a360 = np.array(WM1a_360)

WM2a_360 = []
for i in range(len(WM2a_arr1)):
    if WM2a_arr1[i]<0:
        a_360_i = 360 + WM2a_arr1[i]
    else:
        a_360_i = WM2a_arr1[i]
    WM2a_360.append(a_360_i)
WM2_a360 = np.array(WM2a_360)

In [9]:
R3_finDF = R3_finT.assign(R3_a360=R3_a360,R3_b=R3b_arr1,R3_Sb=R3Sb_arr1)
RMY_finDF = RMY_finT.assign(RMY_a360=RMY_a360,RMY_b=RMYb_arr1,RMY_Sb=RMYSb_arr1)
WM1_finDF = WM1_finT.assign(WM1_a360=WM1_a360,WM1_b=WM1b_arr1,WM1_Sb=WM1Sb_arr1)
WM2_finDF = WM2_finT.assign(WM2_a360=WM2_a360,WM2_b=WM2b_arr1,WM2_Sb=WM2Sb_arr1)

In [10]:
os.chdir(r"F:\benchTest_data2")
R3_finDF.to_csv("R3_meanData_withOriginalUVW_CSAT_TEST_ALL.csv")
RMY_finDF.to_csv("RMY_meanData_withOriginalUVW_CSAT_TEST_ALL.csv")
WM1_finDF.to_csv("WM1_meanData_withOriginalUVW_CSAT_TEST_ALL.csv")
WM2_finDF.to_csv("WM2_meanData_withOriginalUVW_CSAT_TEST_ALL.csv")